# Проект по SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и активно начали создавать приложения для тех, кто любит читать.
Компания "Читабокс" купила крупный сервис для чтения книг по подписке. 

**Цель исследования:**

Проанализировать базу данных, в которой находитсяя информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Знакомство с данными 

Создадим функцию, которая будет показывать результат запроса:

In [2]:
def show(query):
    result = pd.io.sql.read_sql(query, con = engine) 
    return result

### books

In [3]:
query = """ SELECT *
            FROM books
            LIMIT 5;
      """
show(query)  

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### authors

In [4]:
query = """ SELECT *
            FROM authors
            LIMIT 5;
        """
show(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### ratings

In [5]:
query = """ SELECT *
            FROM ratings
            LIMIT 5;
        """
show(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### reviews

In [6]:
query = """ SELECT *
            FROM reviews
            LIMIT 5;
        """
show(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### publishers

In [7]:
query = """ SELECT *
            FROM publishers
            LIMIT 5;
        """
show(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Выполнение запросов

**1. Посчитайте, сколько книг вышло после 1 января 2000 года.**

In [8]:
query = """ SELECT COUNT(title) 
            FROM books
            WHERE publication_date >= '2000-01-01';
        """
show(query)

,count
0,821


После 1 января 2000 г.  вышла 821 книга.

**2. Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [27]:
query = """ SELECT book_title,
       reviews_number,
       avg_rating  
FROM (SELECT b.book_id,
             b.title AS book_title,
             COUNT(DISTINCT(rev.review_id)) AS reviews_number                   
      FROM books AS b
      FULL OUTER JOIN reviews AS rev ON b.book_id = rev.book_id
      GROUP BY b.book_id) AS ab
FULL OUTER JOIN (SELECT b.book_id,
                        AVG(rat.rating) AS avg_rating                  
                 FROM books AS b
                 FULL OUTER JOIN ratings AS rat ON b.book_id = rat.book_id
                 GROUP BY b.book_id) AS cd ON ab.book_id = cd.book_id
ORDER BY avg_rating DESC,
         reviews_number DESC;
""" 
show(query)

,book_title,reviews_number,avg_rating
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


**3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.**

In [ ]:
query = """ SELECT p.publisher,
                   COUNT(b.title) 
            FROM books AS b
            INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY COUNT(b.title) DESC
            LIMIT 1;
        """
show(query)

Наибольшее количество книг в библиотеке было выпущено издательством Penguin Books.

**4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;**

In [16]:
query = """ SELECT b.book_id,
                   b.author_id,
                   COUNT(rat.rating_id) AS ratings_number,
                   AVG(rat.rating) AS avg_rating
            FROM books AS b
            INNER JOIN ratings AS rat ON b.book_id = rat.book_id
            GROUP BY b.book_id
            HAVING COUNT(rat.rating) >=50
            ORDER BY COUNT(rat.rating) DESC
        """
show(query)

,book_id,author_id,ratings_number,avg_rating
0,948,554,160,3.662500
1,750,240,88,4.125000
2,673,235,86,3.825581
3,75,106,84,3.678571
4,302,236,82,4.414634
5,299,236,80,4.287500
6,301,236,75,4.186667
7,79,195,74,3.729730
8,722,240,74,4.391892
9,300,236,73,4.246575


In [18]:
query = """ SELECT au.author,
                   AVG(avg_rating) 
            FROM (SELECT b.book_id,
                         b.author_id,
                         COUNT(rat.rating_id) AS ratings_number,
                         AVG(rat.rating) AS avg_rating
                  FROM books AS b
                  INNER JOIN ratings AS rat ON b.book_id = rat.book_id
                  GROUP BY b.book_id
                  HAVING COUNT(rat.rating) >=50
                  ORDER BY COUNT(rat.rating) DESC) AS top
            LEFT OUTER JOIN authors AS au ON top.author_id = au.author_id
            GROUP BY au.author
            ORDER BY AVG(avg_rating) DESC
            LIMIT 5
        """
show(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Самая высокая оценка книг у автора книг о Гарри Поттере - Джоан Роулинг.

**5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [ ]:
query = """ SELECT AVG(reviews_number)
FROM (SELECT username,
                COUNT(rating) AS ratings_number
            FROM ratings 
            GROUP BY username
            HAVING COUNT(rating) > 50) AS rat
LEFT OUTER JOIN (SELECT username,
                COUNT(review_id) AS reviews_number
            FROM reviews 
            GROUP BY username) AS rev ON rat.username=rev.username;
"""
show(query)

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,3. 